# Regression

## Task explanation
lorem ipsum

## Load used car sales dataset (csv)

In [13]:
# modules
import pandas as pd

In [14]:
# read used car sales dataset
vehicles = pd.read_csv('vehicles.csv', nrows=100000)

In [19]:
car_filter = 'ford'
vehicles.loc[vehicles['manufacturer'] == car_filter]

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
31,7316356412,https://auburn.craigslist.org/cto/d/auburn-uni...,auburn,https://auburn.craigslist.org,15000,2013.0,ford,f-150 xlt,excellent,6 cylinders,...,full-size,truck,black,https://images.craigslist.org/00404_l4loxHvdQe...,2013 F-150 XLT V6 4 Door. Good condition. Leve...,NaN,al,32.592000,-85.518900,2021-05-03T14:02:03-0500
44,7315275942,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,30990,2019.0,ford,ranger supercrew xl pickup,good,NaN,...,NaN,pickup,black,https://images.craigslist.org/00202_4Fz2gXkV8p...,Carvana is the safer way to buy a car During t...,NaN,al,32.590000,-85.480000,2021-05-01T09:21:34-0500
47,7314910156,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,34590,2018.0,ford,f150 super cab xl pickup 4d,good,6 cylinders,...,NaN,pickup,white,https://images.craigslist.org/00w0w_cGdvJjETZc...,Carvana is the safer way to buy a car During t...,NaN,al,32.590000,-85.480000,2021-04-30T12:51:31-0500
50,7314811909,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,38990,2020.0,ford,f150 supercrew cab xlt,good,6 cylinders,...,NaN,pickup,NaN,https://images.craigslist.org/00t0t_6t4cNaVZFh...,Carvana is the safer way to buy a car During t...,NaN,al,32.590000,-85.480000,2021-04-30T10:11:00-0500
53,7314584322,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,27990,2020.0,ford,ranger supercab xl pickup,good,NaN,...,NaN,pickup,white,https://images.craigslist.org/01313_9Zne268YnW...,Carvana is the safer way to buy a car During t...,NaN,al,32.590000,-85.480000,2021-04-29T18:11:11-0500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99981,7309393017,https://jacksonville.craigslist.org/ctd/d/jack...,jacksonville,https://jacksonville.craigslist.org,45795,2019.0,ford,f150 lariat 4x4,like new,6 cylinders,...,full-size,truck,white,https://images.craigslist.org/00H0H_9HDJ3aRr9y...,"*** MINT CONDITION *** BRAND NEW 7"" PRO COMP. ...",NaN,fl,30.207288,-81.738969,2021-04-19T14:39:40-0400
99982,7309392140,https://jacksonville.craigslist.org/ctd/d/jack...,jacksonville,https://jacksonville.craigslist.org,49795,2019.0,ford,f150 supercab stx 4x4,excellent,8 cylinders,...,full-size,truck,white,https://images.craigslist.org/00L0L_8CJvI6I8iS...,"*** MINT CONDITION *** BRAND NEW 7"" PRO COMP. ...",NaN,fl,30.207288,-81.738969,2021-04-19T14:38:25-0400
99983,7309388539,https://jacksonville.craigslist.org/ctd/d/jack...,jacksonville,https://jacksonville.craigslist.org,38495,2016.0,ford,f-150 platinum 4x4,excellent,6 cylinders,...,full-size,truck,blue,https://images.craigslist.org/00o0o_aHVdiLxzZ4...,"*** MINT CONDITION *** BRAND NEW 7"" PRO COMP. ...",NaN,fl,30.207288,-81.738969,2021-04-19T14:33:09-0400
99992,7309363346,https://jacksonville.craigslist.org/ctd/d/jack...,jacksonville,https://jacksonville.craigslist.org,33995,2015.0,ford,f150 lariat supercrew 4x4,excellent,8 cylinders,...,full-size,truck,white,https://images.craigslist.org/00a0a_kLDqLvHZbb...,"*** MINT CONDITION *** BRAND NEW 7"" PRO COMP. ...",NaN,fl,30.207288,-81.738969,2021-04-19T13:57:58-0400
